In [3]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import re
import sys
from collections import defaultdict, deque
from typing import List
import re
from typing import Any, List
import ipydagred3
import networkx as nx
import numpy as np
import openai
from tqdm import tqdm
from CHAMP_utils import *

#os.environ["http_proxy"] = "http://localhost:7890"
#os.environ["https_proxy"] = "http://localhost:7890"
setOpenAi(keyid = 4)

GPT_MODEL = "gpt-4-turbo"
GPT_MODEL = "gpt-4o-mini"
GPT_MODEL = "gpt-3.5-turbo"

# 读取问题数据文件
file_path = '/Users/natehu/Desktop/Tsinghua Research/ToT复现/TOT/champ_116.json'
with open(file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

In [4]:
dataset

[{'ids': 0,
  'problem_identifier': 'P_Combinatorics_3',
  'problem_text': 'On a chess board, two rooks are placed peacefully if they are not on the same row or column. For an n x n chess board, find the number of ways that n rooks can be placed peacefully (i.e., any two are placed peacefully) and the placing is also invariant to a 180-degree rotation.',
  'problem_answer': '2^floor(n/2)*floor(n/2)!'},
 {'ids': 1,
  'problem_identifier': 'P_Combinatorics_7',
  'problem_text': 'Consider a row of 11 seats. A child sits on each. Each child may move by at most one seat. How many possible rearrangements are there (including the original one)?',
  'problem_answer': '144'},
 {'ids': 2,
  'problem_identifier': 'P_Combinatorics_8',
  'problem_text': 'How many strings of length 5 and consisting solely of the digits 1, 2, 3 and 4 have the property that any 1 and 2 are never neighbors (i.e., the string does not contain "12" or "21" as substrings).',
  'problem_answer': '634'},
 {'ids': 3,
  'probl

In [11]:
def split_string_by_(string):
    return string.split('_')[1]

In [12]:
split_string_by_(dataset[60]['problem_identifier'])

'Number-Theory'

In [14]:
i = 0

question = dataset[i]['problem_text']
type = split_string_by_(dataset[i]['problem_identifier'])
gold_answer = dataset[i]['problem_answer']


In [15]:
question

'On a chess board, two rooks are placed peacefully if they are not on the same row or column. For an n x n chess board, find the number of ways that n rooks can be placed peacefully (i.e., any two are placed peacefully) and the placing is also invariant to a 180-degree rotation.'

任务分解 

In [16]:
def decompose_sql(question, type):   

    prompt_for_decompose = f"""

I will now give you a math problem. The type of problem is {type}. Please help me translate this math problem into a series of step-by-step sub-problems.

1 examples are as follows:
Question: Four years ago, Kody was only half as old as Mohamed. If Mohamed is currently twice 30 years old, how old is Kody? 
Answer: 
To solve the question "How old is Kody?", we need to solve the following problems step by step:
1. How old is Mohamed now?
2. How old was Mohamed four years ago?
3. How old was Kody four years ago?
4. How old is Kody now?

Now the command is {question}, please decompose it into a series of easy-to-solve steps like the examples.
Answer Format: (Please write each broken-down question step on a separate line, starting with a number.)
To solve the question "xxx", we need to solve the following problems step by step:
1. sub-question 1
2. sub-question 2
3. sub-question 3
...
"""

    Q = {
        "role": "user",
        "content": prompt_for_decompose
    }
    # Query = Example+[Q]
    Query = [Q]
    result = askChatGPT(Query, model=GPT_MODEL, temperature=1)
    return result


def convert_steps_to_format(decom_commands):
    # 截取“we need to know:”后的内容
    start_index = decom_commands.find("we need to solve the following problems step by step:") + len("we need to solve the following problems step by step:")
    subtasks_text = decom_commands[start_index:].strip()
    # 将每个子任务单独列出
    subtasks = subtasks_text.split('\n')
    subtasks = [task.strip().split('. ', 1)[-1] for task in subtasks]
    steps_dict = {index: value for index, value in enumerate(subtasks)}
    return subtasks, steps_dict

decompose_steps = decompose_sql(question, type)
steps, steps_dict = convert_steps_to_format(decompose_steps)
num_steps = len(steps)
print(steps)
print(steps_dict)  # 只是加了一个问题的编号而已.

total tokens: 471
['How many ways can the first rook be placed on the chess board?', 'How many ways can the second rook be placed on the chess board, ensuring it is peacefully placed with the first rook?', 'How many ways can the third rook be placed on the chess board, ensuring it is peacefully placed with the first two rooks?', 'Continue this process until all n rooks have been placed on the chess board.', 'Determine how this placement is invariant to a 180-degree rotation.', 'Calculate the total number of ways n rooks can be placed peacefully on an n x n chess board.']
{0: 'How many ways can the first rook be placed on the chess board?', 1: 'How many ways can the second rook be placed on the chess board, ensuring it is peacefully placed with the first rook?', 2: 'How many ways can the third rook be placed on the chess board, ensuring it is peacefully placed with the first two rooks?', 3: 'Continue this process until all n rooks have been placed on the chess board.', 4: 'Determine how

In [17]:
print(len(steps))

6


In [18]:
# TOT 求解
N = 3  # 每个子问题进行N次proposal
M = 2  # 通过评估选出M个最好的proposal

solution = []

for i in range(num_steps):
    subtask = steps[i]
    sys_q = f"""
   There is a math_problem. I need you to solve it and give an answer.
Here is the problem:\n{question}

I have broken this math problem down into a series of smaller problems. I will assign you sub-problems one by one, and provide the results of the previous sub-problems as a reference for your reasoning.
Please solve the problem and respond according to mathematical logic.
"""  

    subask = f"""\nThe sub-problem to solve now is xxx: {subtask}
Based on the information above, please provide a concise and clear answer"""
    
    if len(solution)==0:
        # 第一个子问题
        query = subask
        Q = [{'role':'system', 'content':sys_q},
            {'role':'user', 'content':query},]
        for n in range(N):  # 一个子问题提问N次,获取N个解
            result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
            eval_Q = Q + [{'role':'assistant', 'content':result}]
            eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
            score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
            score = int(score)
            
            solution.append((score, [result]))  # 维护一整条推理路径
            
        solution = sorted(solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝
    else:
        temp_solution = []
        for m in range(M):  # 因为剪枝动态维护M个推理路径
            answersSoFar = f"""\nSo far, the answers to the preceding sub-problems are as follows: The format is Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx."""
            for index, value in enumerate(solution[m][1]):
                try:
                    answersSoFar += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""
                except:
                    print('warning')
                    print(index)
                    print(len(solution[m][1]))
                    print(len(steps))
                    sys.exit(0)
            query = answersSoFar+subask
            Q = [{'role':'system', 'content':sys_q},
                 {'role':'user', 'content':query},]
            for n in range(N):  # 一个子问题提问N次,获取N个解
                result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
                eval_Q = Q + [{'role':'assistant', 'content':result}]
                eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
                score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
                score = int(score)
                
                temp_solution.append((solution[m][0]+score, solution[m][1]+[result]))  # 路径score累加
        
        # print(len(temp_solution))  # 此时temp_solution中应该有M*N种推理路径
        solution = sorted(temp_solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝 M*N->M

print(len(solution))
printSeq(solution)
# 用额外的一次query再问一下最终的答案


total tokens: 220
total tokens: 256
total tokens: 218
total tokens: 254
total tokens: 216
total tokens: 252
total tokens: 368
total tokens: 404
total tokens: 399
total tokens: 435
total tokens: 376
total tokens: 412
total tokens: 381
total tokens: 417
total tokens: 374
total tokens: 410
total tokens: 446
total tokens: 482
total tokens: 553
total tokens: 589
total tokens: 546
total tokens: 582
total tokens: 620
total tokens: 656
total tokens: 485
total tokens: 521
total tokens: 485
total tokens: 521
total tokens: 485
total tokens: 521
total tokens: 696
total tokens: 732
total tokens: 757
total tokens: 793
total tokens: 932
total tokens: 968
total tokens: 757
total tokens: 793
total tokens: 792
total tokens: 828
total tokens: 790
total tokens: 826
total tokens: 895
total tokens: 931
total tokens: 993
total tokens: 1029
total tokens: 862
total tokens: 898
total tokens: 902
total tokens: 938
total tokens: 1090
total tokens: 1126
total tokens: 999
total tokens: 1035
total tokens: 1053
total

In [19]:
# 从M个路径里挑一个最好的来问,也可以问完之后再评估一下选最好的答案

user_q = f"""There is a math_problem:\n{question}

I have broken this math problem down into a series of smaller problems and each sub-problem is solved.
The sub-problems and their corresponding answers are as follows. (Format: Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx.)"""

for index, value in enumerate(solution[0][1]):  # 这里仅仅使用了最终得分最高的1条路径来总结得到final answer
    user_q += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""

Q.append({'role':'user', 'content':f"""Now that all the sub-problems have been solved, so what is the final answer?
Please give the final answer without any additional explanation or clarification."""})
finalResult = askChatGPT(Q, model=GPT_MODEL, temperature=1)

# 让大语言模型来判断有没有回答正确
judgeAnswer = {'role':'user', 'content':f"""Here is a math problem with a standard answer and a student's solution. Please help me determine if the student's solution is correct.
Problem: {question}

Standard answer: {gold_answer}

Answer: {finalResult}

If the student's answer is correct, just output True; otherwise, just output False.
No explanation is required.
"""}

Q_judge = [judgeAnswer]
ifcorrect = askChatGPT(Q_judge, model=GPT_MODEL, temperature=1)  # 要么是True, 要么是False

if 'True' in ifcorrect:
    print('correct')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功
elif 'False' in ifcorrect:
    print('error')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功                       

total tokens: 1103
total tokens: 185
error


In [20]:
finalResult

'Sub-problem: Calculate the total number of ways n rooks can be placed peacefully on an n x n chess board.\nAnswer: \\( n^{\\frac{n^2}{4}} \\)'